In [2]:
import math
import json
import random

import tiktoken

from openai import OpenAI

In [772]:
DATA_FILE = "/Users/tddy/Downloads/ChatExport_2024-09-14/result.json"

In [773]:
def check_for_merge(messages: list, start_idx: int, end_idx: int, timeout: int) -> bool:
    first_message, second_to_last_message, last_message = messages[start_idx], messages[end_idx - 1], messages[end_idx]
    if last_message["ts"] - second_to_last_message["ts"] > timeout:
        return False

    if first_message["user_id"] != last_message["user_id"]:
        return False

    return True

In [774]:
def merge_messages(messages: list, start_idx: int, end_idx: int) -> dict:
    texts = [messages[idx]["text"] for idx in range(start_idx, end_idx)]

    merged_message = {}
    merged_message["user_id"] = messages[start_idx]["user_id"]
    merged_message["user_name"] = messages[start_idx]["user_name"]
    merged_message["ts"] = messages[start_idx]["ts"]
    merged_message["id"] = messages[start_idx]["id"]
    merged_message["text"] = ". ".join(texts)

    return merged_message

In [775]:
def merge_consecutive_messages(messages: list, timeout: int) -> list:
    start_idx, end_idx = 0, 0
    merged_messages = []
    while start_idx < len(messages):
        end_idx = end_idx + 1
        if end_idx == len(messages):
            merged_messages.append(merge_messages(messages, start_idx, end_idx))
            start_idx = end_idx
        else:
            start_message, end_message = messages[start_idx], messages[end_idx]
            if not check_for_merge(messages, start_idx, end_idx, timeout):
                merged_messages.append(merge_messages(messages, start_idx, end_idx))
                start_idx = end_idx
    return merged_messages

In [776]:
def read_messages(file_name: str, timeout: int=30) -> list:
    messages = []
    with open(file_name) as file:
        data = json.load(file)
        file.close()
        for record in data["messages"]:
            if record["type"] == "message" and type(record["text"]) is str and len(record["text"]) != 0:
                message = {}
                message["id"] = record["id"]
                message["text"] = record["text"]
                message["ts"] = int(record["date_unixtime"])
                message["user_id"] = record["from_id"]
                message["user_name"] = record["from"]
                messages.append(message)

    messages = sorted(messages, key=lambda x: x["ts"])

    return merge_consecutive_messages(messages, timeout)

In [777]:
messages = read_messages(DATA_FILE, 60)

In [778]:
messages[0:10]

[{'user_id': 'user131782652',
  'user_name': 'Alexander Sobol',
  'ts': 1630613015,
  'id': -999010621,
  'text': 'Добавьте свята у меня нет его телеги'},
 {'user_id': 'user131782652',
  'user_name': 'Alexander Sobol',
  'ts': 1630613634,
  'id': -999010617,
  'text': 'Чтобы вам было проще с меня 7\xa0140 (фальцетом)'},
 {'user_id': 'user131782652',
  'user_name': 'Alexander Sobol',
  'ts': 1630613786,
  'id': -999010615,
  'text': 'Блять там в чеке ещё ндс'},
 {'user_id': 'user237923972',
  'user_name': 'Dmitrii Ushanov',
  'ts': 1630613814,
  'id': -999010614,
  'text': 'не саш. это оч много. давай в банке сочтемся. как там с вакансиями, кстати?'},
 {'user_id': 'user218280715',
  'user_name': 'Evgeniya Ponomarenko',
  'ts': 1630613844,
  'id': -999010610,
  'text': 'Можно микрокредит взять до зп?'},
 {'user_id': 'user278782876',
  'user_name': 'Борис Шарчилев',
  'ts': 1630613949,
  'id': -999010607,
  'text': 'Я не хочу платить'},
 {'user_id': 'user131782652',
  'user_name': 'Alexan

In [779]:
class ChatDataset:
    def __init__(self, messages: list, max_context_tokens: int=5000, max_context_messages: int=5, gpt_model_name: str="gpt-4o"):
        self.messages = messages
        
        self.max_context_tokens = max_context_tokens
        self.max_context_messages = max_context_messages
        
        self.tokenizer = tiktoken.encoding_for_model(gpt_model_name)
   
    def __len__(self) -> int:
        return len(self.messages)

    def __getitem__(self, idx: int) -> dict:
        context_length = random.randint(1, self.max_context_messages)
        bot_message = self.messages[idx]

        result = {}
        result["messages"] = []
        result["messages"].append({"role": "system", "content": "Продолжи цепочку сообщений в групповом чате наиболее подходящим образом."})
        context_idx = idx - 1
        token_count = 0
        while len(result["messages"]) < context_length and context_idx >= 0:
            text = messages[context_idx]["user_name"] + ": " + messages[context_idx]["text"]
            token_count += len(encoding.encode(text))
            if token_count > self.max_context_tokens:
                break

            result["messages"].append({"role": "user", "content": text})
            context_idx -= 1

        result["messages"].append({"role": "assistant", "content": bot_message["text"], "weight": 1})

        return result

    def get_users(self) -> set:
        return self.user_to_message_ids.keys()

In [780]:
dataset = ChatDataset(messages, max_context_tokens=5000, max_context_messages=30)

In [781]:
dataset[53]

{'messages': [{'role': 'system',
   'content': 'Продолжи цепочку сообщений в групповом чате наиболее подходящим образом.'},
  {'role': 'user', 'content': 'Dmitrii Ushanov: овербукинг на позицию'},
  {'role': 'user', 'content': 'Борис Шарчилев: Мои позиции ослабляются'},
  {'role': 'user',
   'content': 'Svyatoslav Yushin: Понятно почему на кухне больше черешни нет. Уже тогда всё было решено...'},
  {'role': 'user',
   'content': 'Dmitrii Ushanov: не забудь подчеркнуть сочетание рубашки и штанов'},
  {'role': 'user',
   'content': 'Борис Шарчилев: Мне кстати сео в слаке писал, но так и не поставил'},
  {'role': 'user',
   'content': 'Evgeniya Ponomarenko: Ахаха. Про отпуск так и не спросили, завтра спрошу сео'},
  {'role': 'user',
   'content': 'Dmitrii Ushanov: и вопросы оттуда задаете!!!111'},
  {'role': 'user',
   'content': 'Svyatoslav Yushin: Поездка будет быстрой, простой поездка не будет'},
  {'role': 'user',
   'content': 'Evgeniya Ponomarenko: Ну мы люди простые. В свою жопу по

In [792]:
TRAIN_DATASET_FILE = "/Users/tddy/Downloads/ChatExport_2024-09-14/train_dataset_large"
TEST_DATASET_FILE = "/Users/tddy/Downloads/ChatExport_2024-09-14/test_dataset_large"

In [793]:
def num_tokens_from_messages(samples, model="gpt-4o-mini"):
    encoding = tiktoken.encoding_for_model(model)
    num_tokens = 0
    for sample in samples:
        for message in sample["messages"]:
            num_tokens += 4
            for key, value in message.items():
                if key != "weight":
                    num_tokens += len(encoding.encode(value))
        num_tokens += 2
    return num_tokens

In [794]:
def split_train_test(dataset: ChatDataset, train_output_path: str, test_output_path:str, sample_rate: float, test_share: float=0.1):
    split_idx = len(dataset) - int(len(dataset) * test_share)

    samples = []
    for idx in range(0, split_idx):
        if random.random() < sample_rate:
            sample = dataset[idx]
            samples.append(sample)  
    print("Train tokens: " + str(num_tokens_from_messages(samples)))

    with open(train_output_path, "w") as file:
        file.write("\n".join([json.dumps(sample, ensure_ascii=False) for sample in samples]))

    samples = []
    for idx in range(split_idx, len(dataset)):
        if random.random() < sample_rate:
            sample = dataset[idx]
            samples.append(sample)
    print("Test tokens: " + str(num_tokens_from_messages(samples)))

    with open(test_output_path, "w") as file:
        file.write("\n".join([json.dumps(sample, ensure_ascii=False) for sample in samples]))

In [795]:
split_train_test(dataset, TRAIN_DATASET_FILE, TEST_DATASET_FILE, 0.15, 0.05)

Train tokens: 15269733
Test tokens: 730484


In [7]:
API_KEY = ""
ORGANIZATION_ID = ""

In [4]:
client = OpenAI(api_key=API_KEY, organization=ORGANIZATION_ID)

In [801]:
ret = client.files.create(
  file=open(TRAIN_DATASET_FILE, "rb"),
  purpose="fine-tune"
)

In [803]:
ret

FileObject(id='file-o2MN2ORowM72Uy0gJE1tBQxW', bytes=81570505, created_at=1727028465, filename='train_dataset_large', object='file', purpose='fine-tune', status='processed', status_details=None)

In [799]:
ret = client.files.create(
  file=open(TEST_DATASET_FILE, "rb"),
  purpose="fine-tune"
)

In [800]:
ret

FileObject(id='file-ukU9i6ipCenAPzPHX7L6NWX9', bytes=3988526, created_at=1727028281, filename='test_dataset_large', object='file', purpose='fine-tune', status='processed', status_details=None)

In [804]:
ret = client.fine_tuning.jobs.create(
    training_file="file-o2MN2ORowM72Uy0gJE1tBQxW",
    validation_file="file-ukU9i6ipCenAPzPHX7L6NWX9",
    model="gpt-4o-mini-2024-07-18",
    hyperparameters={"n_epochs":2, "batch_size": 64, "learning_rate_multiplier":0.1}
)

In [805]:
ret

FineTuningJob(id='ftjob-bjhoGI05AJ0Q7x6u1VLsnvEI', created_at=1727028531, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=2, batch_size=64, learning_rate_multiplier=0.1), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-vHh861blWyZBvSmugtU4QmIU', result_files=[], seed=1079764368, status='validating_files', trained_tokens=None, training_file='file-o2MN2ORowM72Uy0gJE1tBQxW', validation_file='file-ukU9i6ipCenAPzPHX7L6NWX9', estimated_finish=None, integrations=[], user_provided_suffix=None)

In [6]:
client.fine_tuning.jobs.retrieve("ftjob-bjhoGI05AJ0Q7x6u1VLsnvEI")

FineTuningJob(id='ftjob-bjhoGI05AJ0Q7x6u1VLsnvEI', created_at=1727028531, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=2, batch_size=64, learning_rate_multiplier=0.1), model='gpt-4o-mini-2024-07-18', object='fine_tuning.job', organization_id='org-vHh861blWyZBvSmugtU4QmIU', result_files=[], seed=1079764368, status='validating_files', trained_tokens=None, training_file='file-o2MN2ORowM72Uy0gJE1tBQxW', validation_file='file-ukU9i6ipCenAPzPHX7L6NWX9', estimated_finish=None, integrations=[], user_provided_suffix=None)

In [660]:
response = client.chat.completions.create(
  model="ft:gpt-4o-mini-2024-07-18:personal::AAMiVZbj",
  messages=[
    {"role": "system", "content": "Продолжи цепочку сообщений в групповом чате наиболее подходящим образом."},
    {"role": "user", "content": "Evgeniya Ponomarenko: Уверена кстати что нихуя не сделаете из-за Саши"},
    {"role": "user", "content": "Evgeniya Ponomarenko: Он затянет в болото, а потом бросит, как все свои увлечения"},
    {"role": "user", "content": "Alexander Sobol: Ого какой гной"},
    {"role": "user", "content": "Evgeniya Ponomarenko: без гавна"},
    {"role": "user", "content": "Dmitrii Ushanov: ого, а почему заорала?"},
    {"role": "user", "content": "Dmitrii Ushanov: это пока похоже на развитие событий"},
    {"role": "user", "content": "Dmitrii Ushanov: но я нашел саше мотивацию"},
    {"role": "user", "content": "Dmitrii Ushanov: такой же стартап делает витекv"},
    {"role": "user", "content": "Dmitrii Ushanov: так что пока вроде он заинтересован"},
    {"role": "user", "content": "Evgeniya Ponomarenko: Типа его конкуренция сможешь мотивировать с дивана встать?"},
    {"role": "user", "content": "Dmitrii Ushanov: не, типа прорабатываем стратегию второго шанса"},
    {"role": "user", "content": "Evgeniya Ponomarenko: Ну нужно сильно мотивировать"},
    {"role": "user", "content": "Dmitrii Ushanov: так а че саша совсем стал не дееспособный?"},
    {"role": "user", "content": "Dmitrii Ushanov: да он только на диване может лежать и ничего не делать"},
    {"role": "user", "content": "Alexander Sobol: виталик, как дела в буках?"}
  ]
)

In [661]:
response

ChatCompletion(id='chatcmpl-AAFQIfy2HJvSemn26uDTMmQG2SZDw', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Vitaly Grigorash. Ну хз. Мы в плюсах', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1727005930, model='ft:gpt-4o-2024-08-06:personal::AAFLBKbW', object='chat.completion', service_tier=None, system_fingerprint='fp_845b2ae0fe', usage=CompletionUsage(completion_tokens=14, prompt_tokens=45, total_tokens=59, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))